In [1]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
with open("config_tests.yaml", "r") as stream:
    config_tests = yaml.safe_load(stream)

In [3]:
N_EPOCHS = 50
N_TRIALS = 50
BATCH_SIZE = 512

CONFIG = config_tests["random"]

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [4]:
x, y = utils.load_cifar_data("../data/cifar-10-batches-py/")
x /= 255

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42, stratify=y)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]

In [5]:
train_size, len(x_val), len(x_test)

(30000, 10000, 10000)

In [6]:
assessment_model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["assessment_model"])

assessment_model.compile(loss=loss, metrics=["accuracy"])
assessment_model(x_train[:1])
assessment_model.load_weights("../models/assessment_model.weights.h5")

2024-03-27 15:24:16.778497: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-03-27 15:24:16.778520: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-03-27 15:24:16.778525: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-03-27 15:24:16.778543: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-27 15:24:16.778555: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/dbugajny/PycharmProjects/CurriculumLearning/venv/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop

In [7]:
assessment_model.evaluate(x_test, y_test)

2024-03-27 15:24:17.233687: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5315 - loss: 1.3379


[1.3303395509719849, 0.531499981880188]

In [ ]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

samples_proba = utils.calculate_proba2(
    assessment_model, x_train_sorted, y_train_sorted, counts
)

In [ ]:
model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["test_model"])
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
model(x_train[0:1])
### model.save_weights("../models/default_model.weights.h5")
model.load_weights("../models/default_model.weights.h5")
model_weights = model.get_weights()

In [ ]:
model_scores = []

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)
        
    for i in range(N_EPOCHS):
        n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)
        
        samples_ids = utils.chose_samples(n_samples, samples_proba, CONFIG["order_type"])
        model.fit(
            x_train_sorted[samples_ids],
            y_train_sorted[samples_ids],
            # validation_data=(x_val, y_val),
            epochs=1,
            batch_size=BATCH_SIZE,
            verbose=0,
        )
        
        if CONFIG["progressive"]:
            samples_proba = utils.calculate_proba2(
                model, x_train_sorted, y_train_sorted, counts # , CONFIG["negative_loss"]
            )
            
    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
    model_scores.append(accuracy)
    print("Mean:", np.mean(model_scores))

In [ ]:
np.mean(model_scores)

In [ ]:
np.median(model_scores)

In [ ]:
np.mean(model_scores)

In [ ]:
np.median(model_scores)

In [ ]:
abc = [0.5891333222389221,
 0.6179333329200745,
 0.6141999959945679,
 0.5941333174705505,
 0.5335333347320557,
 0.6105333566665649,
 0.629800021648407,
 0.5806000232696533,
 0.607200026512146,
 0.5431333184242249,
 0.6028000116348267,
 0.5551999807357788,
 0.5738000273704529,
 0.5960000157356262,
 0.6377333402633667,
 0.6040666699409485,
 0.602733314037323,
 0.5839999914169312,
 0.5225333571434021,
 0.548466682434082,
 0.630466639995575,
 0.6215999722480774,
 0.614466667175293,
 0.6194000244140625,
 0.6003999710083008,
 0.6016666889190674,
 0.5498666763305664,
 0.6236666440963745,
 0.5487333536148071,
 0.6291999816894531,
 0.6353999972343445,
 0.5995333194732666,
 0.5789999961853027,
 0.6046666502952576,
 0.6092000007629395,
 0.4715999960899353,
 0.6051333546638489,
 0.6222666501998901,
 0.38839998841285706,
 0.5794666409492493,
 0.6154000163078308,
 0.6182666420936584,
 0.5651999711990356,
 0.5130666494369507,
 0.6120666861534119,
 0.6335333585739136,
 0.4111333191394806,
 0.5278000235557556,
 0.5985333323478699,
 0.6223999857902527]

In [ ]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 6))
# plt.subplot(131)
# plt.boxplot(results["test_model_1"])
# plt.ylim([0.63, 0.65])
# plt.subplot(132)
# plt.boxplot(ass_m["test_model_1"])
# plt.ylim([0.63, 0.65])
# plt.subplot(133)
# plt.boxplot(random_m["test_model_1"])
# plt.ylim([0.63, 0.65])
# plt.show()

In [ ]:
import scipy.stats
scipy.stats.ttest_ind(abc * 2, model_scores * 2)

In [ ]:
abc = [0.6462666392326355,
 0.6143333315849304,
 0.6458666920661926,
 0.6164666414260864,
 0.5498666763305664,
 0.659333348274231,
 0.5580000281333923,
 0.6102666854858398,
 0.6272666454315186,
 0.5816666483879089,
 0.5935333371162415,
 0.6465333104133606,
 0.6446666717529297,
 0.6413333415985107,
 0.6402666568756104,
 0.6539333462715149,
 0.627133309841156,
 0.5664666891098022,
 0.6363333463668823,
 0.5622000098228455,
 0.5845999717712402,
 0.6078000068664551,
 0.6403999924659729,
 0.5857333540916443,
 0.60753333568573,
 0.6057999730110168,
 0.5952666401863098,
 0.6287333369255066,
 0.6296666860580444,
 0.598800003528595,
 0.6316666603088379,
 0.6499999761581421,
 0.6220666766166687,
 0.5823333263397217,
 0.6305333375930786,
 0.5469333529472351,
 0.6465333104133606,
 0.623199999332428,
 0.6430000066757202,
 0.6377333402633667,
 0.5971999764442444,
 0.6179333329200745,
 0.6143333315849304,
 0.584933340549469,
 0.4405333399772644,
 0.6126000285148621,
 0.5825999975204468,
 0.6133333444595337,
 0.6164666414260864,
 0.5774666666984558]
np.median(abc)